# 5.2 Evaluating machine-learning models

In the three examples presented in chapter 4, we split the data into a training set, a
validation set, and a test set. The reason not to evaluate the models on the same data
they were trained on quickly became evident: after just a few epochs, all three models
began to *overfit*. That is, their performance on never-before-seen data started stalling
(or worsening) compared to their performance on the training data—which always
improves as training progresses.

 In machine learning, the goal is to achieve models that *generalize*—that perform
well on never-before-seen data—and overfitting is the central obstacle. You can only
control that which you can observe, so it’s crucial to be able to reliably measure the
generalization power of your model. The following sections look at strategies for mitigating overfitting and maximizing generalization. In this section, we’ll focus on how
to measure generalization: how to evaluate machine-learning models.

## 5.2.1 Training, validation, and test sets

Evaluating a model always boils down to splitting the available data into three sets: training, validation, and test. You train on the training data and evaluate your model on the validation data. Once your model is ready for prime time, you test it one final time on the test data.

You may ask, why not have two sets: a training set and a test set? You’d train on the training data and evaluate on the test data. Much simpler!

 The reason is that developing a model always involves tuning its configuration: for example, choosing the number of layers or the size of the layers (called the *hyperparameters* of the model, to distinguish them from the *parameters*, which are the network’s weights). You do this tuning by using as a feedback signal the performance of the model on the validation data. 
 
 In essence, this tuning is a form of *learning*: a search
for a good configuration in some parameter space. As a result, tuning the configuration of the model based on its performance on the validation set can quickly result in *overfitting to the validation set*, even though your model is never directly trained on it.

 Central to this phenomenon is the notion of *information leaks*. Every time you tune a hyperparameter of your model based on the model’s performance on the validation set, some information about the validation data leaks into the model. If you do this only once, for one parameter, then very few bits of information will leak, and your validation set will remain reliable to evaluate the model. But if you repeat this many times—running one experiment, evaluating on the validation set, and modifying your model as a result—then you’ll leak an increasingly significant amount of information about the validation set into the model.

Splitting your data into training, validation, and test sets may seem straightforward,
but there are a few advanced ways to do it that can come in handy when little data is
available. Let’s review three classic evaluation recipes: simple hold-out validation, K-fold validation, and iterated K-fold validation with shuffling.

### *Simple hold-out validation*

Set apart some fraction of your data as your test set. Train on the remaining data, and evaluate on the test set. As you saw in the previous sections, in order to prevent information leaks, you shouldn’t tune your model based on the test set, and therefore you should also reserve a validation set. The following cell shows a simple implementation.

In [ ]:
num_validation_samples = 10000

# Shuffing the data is usually appropriate
np.random.shuffle(data)

# Defines the validation set
validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]

# Defines the training set
training_data = data[:]

# Trains a model on the training data, 
# and evaluates it on the validation data
model = get_model()
model.train(training_data)
validation_score = model.evaluation(validation_data)

# At this point you can tune your model, 
# retrain it, evaluate it, tune it again...
...

# Once you've turned your hyperparameters, it's common to train 
# your final models from scratch on all non-test data available.
model = get_model()
model.train( np.concatenate([training_data, validation_data]) )
test_score = model.evaluate(test_data)

This is the simplest evaluation protocol, and it suffers from one flaw: if little data is
available, then your validation and test sets may contain too few samples to be statistically representative of the data at hand. This is easy to recognize: if different random
shuffling rounds of the data before splitting end up yielding very different measures
of model performance, then you’re having this issue. K-fold validation and iterated
K-fold validation are two ways to address this, as discussed next.

### *K-fold validation*

With this approach, you split your data into *K* partitions of equal size. For each partition `i`, train a model on the remaining *K* – 1 partitions, and evaluate it on partition `i`. Your final score is then the averages of the *K* scores obtained. This method is helpful when the performance of your model shows significant variance based on your traintest split. Like hold-out validation, this method doesn’t exempt you from using a distinct validation set for model calibration.

In [ ]:
k = 4
num_validation_samples = len(data) // k
np.random.shuffle(data)

validation_scores = []
for fold in range(k):
    # Selects the validation-data partition
    validation_data = data[num_validation_samples * fold:
                          num_validation_samples * (fold + 1)]
    # Uses the remainder of the data as training data
    training_data = np.concatenate(data[:num_validation_samples * fold], 
                                   data[num_validation_samples * (fold + 1):]) 

    
    # Creates a brand-new instance of the model (untrained)
    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)
    
# Validation score: average of the validation scores of the k folds
validation_score = np.average(validation_scores)

# Train the final model on all non-test data available
model = get_model()
model.train(data)
test_score = model.evaluate(test_data)

### *Iterated K-fold validation with shuffling*

This one is for situations in which you have relatively little data available and you need
to evaluate your model as precisely as possible. I’ve found it to be extremely helpful in Kaggle competitions. It consists of applying K-fold validation multiple times, shuffling the data every time before splitting it *K* ways. The final score is the average of the scores obtained at each run of K-fold validation. Note that you end up training and evaluating *P* × *K* models (where *P* is the number of iterations you use), which can very expensive.

## 5.2.2 Beating the common-sense baseline
## 5.2.3 Things to keep in mind

Keep an eye out for the following when you’re choosing an evaluation protocol:
- *Data representativeness*—You want both your training set and test set to be representative of the data at hand. For instance, if you’re trying to classify images of digits, and you’re starting from an array of samples where the samples are ordered by their class, taking the first 80% of the array as your training set and the remaining 20% as your test set will result in your training set containing only classes 0–7, whereas your test set contains only classes 8–9. This seems like a ridiculous mistake, but it’s surprisingly common. For this reason, you usually should *randomly shuffle* your data before splitting it into training and test sets.
- *The arrow of time*—If you’re trying to predict the future given the past (for example, tomorrow’s weather, stock movements, and so on), you should *not* randomly shuffle your data before splitting it, because doing so will create a *temporal leak*: your model will effectively be trained on data from the future. In such situations, you should always make sure all data in your test set is *posterior* to the data in the training set.
- *Redundancy in your data*—If some data points in your data appear twice (fairly common with real-world data), then shuffling the data and splitting it into a training set and a validation set will result in redundancy between the training and validation sets. In effect, you’ll be testing on part of your training data, which is the worst thing you can do! Make sure your training set and validation set are disjoint.